In [2]:
#@title <b><font color="red">▶</font><font color="black"> run this cell to prepare supplementary materials for the lesson</font></b>

!rm -rf harbour-space-text-mining-course
!git clone https://github.com/horoshenkih/harbour-space-text-mining-course.git
import sys
sys.path.append('harbour-space-text-mining-course')

from tmcourse.ipyquiz import Quiz

from tmcourse.utils import (
    enable_mathjax_in_cell,
    display_cv_results,
    display_token_importance,
)
from tmcourse.quiz import (
    quiz_conditional_probability,
    quiz_chain_rule,
    quiz_bigram_lm,
    quiz_count_ngrams,
    quiz_perplexity,
    quiz_random_benchmark,
    quiz_pipeline_parameter,
)
from tmcourse.demo import (
    demo_generate_text_ngram,
)

from collections import Counter
from math import exp
from tabulate import tabulate
from tqdm.notebook import tqdm
from IPython.display import HTML, display

Cloning into 'harbour-space-text-mining-course'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 424 (delta 83), reused 77 (delta 44), pack-reused 307
Receiving objects: 100% (424/424), 40.03 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (266/266), done.


<!--@slideshow slide-->
# Language Models

<!--@slideshow slide-->
# Last lesson's review
1. `spaCy`
  - tokenization
  - lemmatization
  - NER
2. TF-IDF
  - measures informativity of a term in a document
  - the product of Term Frequency (TF) and Inverse Document Frequency (IDF)
3. Vectorizers in `sklearn`

<!--@slideshow slide-->
# Plan for today
1. Probability refresher: conditional probability and chain rule
2. Definition of Language Model
3. Algorithm: $n$-gram Language Model
4. Generate text with Language Model
5. Evaluate quality of Language Model
6. Text classification with Language Models

<!--@slideshow slide-->
# Probability refresher


<!--@slideshow slide-->
## Conditional probability
Probability of $B$ given $A$ is defined as
$$
\Pr(B|A) = \dfrac{\Pr(AB)}{\Pr(A)}
$$

**Interpretation**: instead of full probability space, consider the subspace where event $A$ occurs.

**Example**: Compute $\Pr(\textrm{''great''}|\textrm{''america''})$ in tweets of Donald Trump.
> How likely does Trump say "great" when he says "america"?

In [0]:
import json
with open("harbour-space-text-mining-course/datasets/trump_twitter_archive/tweets.json") as f:
    tweets = json.load(f)

<!--@slideshow slide-->
Compute $\Pr(\textrm{''great''}|\textrm{''america''})$ using the definition of conditional probability.

In [4]:
#@slideshow fragment
total_tweets = len(tweets)
tweets_with_america = sum([
    "america" in t["text"].lower()
    for t in tweets
])
tweets_with_america_great = sum([
    "america" in t["text"].lower() and "great" in t["text"].lower()
    for t in tweets
])
# P(A)
P_america = tweets_with_america / total_tweets
# P(AB)
P_america_great = tweets_with_america_great / total_tweets
# P(B|A)
P_great_given_america = P_america_great / P_america

print("Total tweets:", total_tweets)
print('# times "america" occurs in tweets:', tweets_with_america)
print('# times "america" and \"great\" occur in tweets:', tweets_with_america_great)
print('P("america") =', P_america)
print('P("america" "great") =', P_america_great)
print('P("great" | "america") =', P_great_given_america)

Total tweets: 48040
# times "america" occurs in tweets: 4289
# times "america" and "great" occur in tweets: 1710
P("america") = 0.08927976686094921
P("america" "great") = 0.03559533721898418
P("great" | "america") = 0.39869433434366985


<!--@slideshow slide-->
Interpret the conditional probability $\Pr(B|A)$ as the probability of $B$ in the subspace where $A$ occurs

In [5]:
#@slideshow fragment
# event A = tweet contains "america"
# event B = tweet contains "great"

N = 0
N_B = 0
for t in tweets:
    tweet_text = t["text"].lower()
    if "america" not in tweet_text:
        # ignore all the events where A do not occur
        continue
    # count total events in the subspace
    N += 1
    if "great" in tweet_text:
        # count events B in the subspace
        N_B += 1
print('P("great" | "america") =', N_B / N)

P("great" | "america") = 0.39869433434366985


<!--@slideshow slide-->
## Colab quiz 1

In [6]:
quiz_conditional_probability()()

<!--@slideshow slide-->
## Chain rule
Chain rule is the successive application of the definition of conditional probability.


<!--@slideshow fragment-->
For 3 events $A, B, C$
$$
\Pr(ABC) = \Pr(A) \cdot \Pr(BC|A)
$$

<!--@slideshow fragment-->
Denote $\Pr_A(\cdot) \equiv \Pr(\cdot|A)$. Again, the interpretation is: instead of full probability space, consider the subspace where $A$ occurs.

<center>$\Pr(BC|A) \equiv \Pr_A (BC) = \Pr_A(B) \cdot \Pr_A(C|B)$</center>


<!--@slideshow fragment-->
Substituting to the first equation:
$$
\Pr(ABC) = \Pr(A) \cdot \Pr(BC|A) = \Pr(A) \cdot \Pr(B|A) \cdot \Pr(C|BA)
$$


<!--@slideshow fragment-->
The event $A$ is not special, we can use $B$ instead:
$$
\Pr(ABC) = \Pr(B) \cdot \Pr(AC|B) = \Pr(B) \cdot \Pr(C|B) \cdot \Pr(A|BC)
$$


<!--@slideshow slide-->
General form of chain rule:
$$
\Pr(A_1 A_2 \dots A_n) = \Pr(A_1) \cdot \Pr(A_2 | A_1) \cdot \dots \cdot \Pr(A_n| A_{n-1} A_{n-2} \dots A_2 A_1)
$$

<!--@slideshow slide-->
## Colab quiz 2

In [7]:
quiz_chain_rule()()

<!--@slideshow slide-->
# What is Language Model?

<!--@slideshow fragment-->
**Definition 1**: a _language model_ is an algorithm that predicts (estimates) the probability of a text:

$$
\Pr(\textrm{"never gonna give you up"}) = ?
$$

<!--@slideshow fragment-->
**Definition 2**: a _language model_ is an algorithm that predicts (estimates) the conditional probability of the next word in a sequence:

$$
\Pr(\textrm{"up"} | \textrm{"never gonna give you"}) = ?
$$

> "Predict the conditional probability of the next word" is just a formal way to say "Predict the next word"

<!--@slideshow fragment-->
**These definitions are equivalent!**

<!--@slideshow slide-->
## Definition 1 implies Definition 2

We can get conditional probabilities of words from probabilities of texts, like this:
$$
\Pr(\textrm{"up"} | \textrm{"never gonna give you"}) = \dfrac{\Pr(\textrm{"never gonna give you up"})}{\Pr(\textrm{"never gonna give you"})}
$$
Or, in general
$$
\Pr(t_n | t_1 t_2 \dots t_{n-1}) = \dfrac{\Pr(t_1 t_2 \dots t_{n-1} t_n)}{\Pr(t_1 t_2 \dots t_{n-1})}
$$

<!--@slideshow slide-->
## Definition 2 implies Definition 1

We can compute the probability of a sequence from conditional probabilities of words using chain rule, like this:
$$
\Pr(\textrm{"never gonna give you up"}) = \Pr(\textrm{"never"}) \cdot \Pr(\textrm{"gonna"} | \textrm{"never"}) \cdot \Pr(\textrm{"give"}| \textrm{"never gonna"}) \cdot \Pr(\textrm{"you"}| \textrm{"never gonna give"}) \cdot \Pr(\textrm{"up"} | \textrm{"never gonna give you"})
$$
Or, in general
$$
\Pr(t_1t_2\dots t_n) = \Pr(t_1) \cdot \Pr(t_2 | t_1) \cdot \Pr(t_3 | t_1 t_2) \cdot \dots \cdot \Pr(t_n | t_1 t_2 \dots t_{n-1})
$$

<!--@slideshow slide-->
**Q**: How to estimate the probability of a sequence?


<!--@slideshow fragment-->
**A**: Count how often the sequence occurs in data.


<!--@slideshow fragment-->
**Q**: But the data is finite, and the number of sequences is infinite! Is that possible?


<!--@slideshow fragment-->
**A**: Well, we need an approximation.

<!--@slideshow slide-->
Assumption (Markov property): the probability of the next token depends only on $\color{red}k$ previous tokens.
$$
\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \Pr(t_i | t_{\color{red}{i-k}} t_{\color{red}{i-k+1}} \dots t_{i-1})
$$
Example for $k=3$
$$
\Pr(\textrm{"up"} | \textrm{"never gonna give you"}) = \Pr(\textrm{"up"} | \textrm{"gonna give you"})
$$


<!--@slideshow slide-->
**Definition**: $n$_-gram_ is a sequence of $n$ tokens.

Special cases:
- 1-gram is called _unigram_
- 2-gram is called _bigram_
- 3-gram is called _trigram_

<!--@slideshow slide-->
**Definition**: _$n$-gram language model_ estimates the probability of a sequence assuming that each token depends on $n-1$ previous tokens.

<!--@slideshow fragment-->
Examples:
- A unigram (1-gram) language model assumes that all the tokens are independent (each token depends on 0 preceeding tokens).
  - $\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \Pr(t_i)$

<!--@slideshow fragment-->
- A bigram (2-gram) language model assumes that the next token depends on the latest preceeding token.
  - $\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \Pr(t_i| t_{i-1}) = \dfrac{\Pr(t_{i-1} t_i)}{\Pr(t_{i-1})}$

<!--@slideshow fragment-->
- For a general $n$-gram language model, it is sufficient to know probabilities of $n$-grams and $n-1$-grams.
  - $\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \Pr(t_i| t_{i-n + 1} \dots t_{i-1}) = \dfrac{\Pr(\overbrace{t_{i-n + 1} \dots t_i}^{n\textrm{ tokens}})}{\Pr(\underbrace{t_{i-n + 1} \dots t_{i-1}}_{n-1\textrm{ tokens}})}$

<!--@slideshow slide-->
## Colab quiz 3

In [8]:
enable_mathjax_in_cell()
quiz_bigram_lm()()

<!--@slideshow slide-->
# Algorithm: $n$-gram Language Model

<!--@slideshow slide-->
To build an $n$-gram language model, we need to compute probabilities of $n$-grams and $n-1$-grams.

To do so, we need to count $n$-grams and $n-1$-grams.

In [9]:
#@slideshow slide
def generate_n_grams(sequence, n):
    # it is convenient to add padding to the beginnning and to the end of the sequence
    # 'None' is the technical token
    padding = [None for _ in range(n-1)]
    # we extract n-grams from the padded sequence
    padded_sequence = padding + sequence + padding
    generated_ngrams = []
    # sliding window of size n: iterate over first n-1 technical tokens, then len(sequence) "real" tokens
    for i in range(n - 1 + len(sequence)):
        # take the slice of size n starting with i-th token
        generated_ngrams.append(tuple(padded_sequence[i:i+n]))

    return generated_ngrams

# look at the examples
unigrams = generate_n_grams(list("abcdef"), 1)
bigrams = generate_n_grams(list("abcdef"), 2)
trigrams = generate_n_grams(list("abcdef"), 3)
print(len(unigrams), "unigrams:", unigrams)
print(len(bigrams), "bigrams:", bigrams)
print(len(trigrams), "trigrams:", trigrams)

6 unigrams: [('a',), ('b',), ('c',), ('d',), ('e',), ('f',)]
7 bigrams: [(None, 'a'), ('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e'), ('e', 'f'), ('f', None)]
8 trigrams: [(None, None, 'a'), (None, 'a', 'b'), ('a', 'b', 'c'), ('b', 'c', 'd'), ('c', 'd', 'e'), ('d', 'e', 'f'), ('e', 'f', None), ('f', None, None)]


In [10]:
#@slideshow fragment
# usage:
from collections import Counter
print(Counter(generate_n_grams(list("aabab"), 2)))

Counter({('a', 'b'): 2, (None, 'a'): 1, ('a', 'a'): 1, ('b', 'a'): 1, ('b', None): 1})


<!--@slideshow slide-->
Remember the formula for $n$-gram language model:
$$
\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \Pr(t_i| t_{i-n + 1} \dots t_{i-1}) = \dfrac{\Pr(\overbrace{t_{i-n + 1} \dots t_i}^{n\textrm{ tokens}})}{\Pr(\underbrace{t_{i-n + 1} \dots t_{i-1}}_{n-1\textrm{ tokens}})}
$$


<!--@slideshow fragment-->
We estimate probabilities by counts:
- $\Pr(t_{i-n} \dots t_{i-1}) = \dfrac{\textrm{count}(t_{i-n} \dots t_{i-1})}{\textrm{total # of }n\textrm{-grams}}$
- $\Pr(t_{i-n + 1} \dots t_{i-1}) = \dfrac{\textrm{count}(t_{i-n + 1} \dots t_{i-1})}{\textrm{total # of }n-1\textrm{-grams}}$


<!--@slideshow fragment-->
Since 
$$\textrm{total # of }n\textrm{-grams} \approx \textrm{total # of }n-1\textrm{-grams}$$

we have
$$
\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \dfrac{\textrm{count}(t_{i-n} \dots t_{i-1})}{\textrm{count}(t_{i-n + 1} \dots t_{i-1})}
$$


<!--@slideshow slide-->
$$
\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \dfrac{\textrm{count}(t_{i-n} \dots t_{i-1})}{\textrm{count}(t_{i-n + 1} \dots t_{i-1})}
$$
Two possible problems:
1. Zero counts in the numerator
1. Zero counts in the denominator


<!--@slideshow fragment-->
Solution: add _smoothing_:
$$
\Pr(t_i | t_1 t_2 \dots t_{i-1}) = \dfrac{\textrm{count}(t_{i-n} \dots t_{i-1}) \color{red}{+\delta}}{\textrm{count}(t_{i-n + 1} \dots t_{i-1})\color{red}{+\delta \cdot |V|}}
$$
where $\delta$ is a small number, and $V$ is the _vocabulary_ (the set of all tokens).


<!--@slideshow fragment-->
**Intuition**: if we don't have enough data to estimate probabilities, any token is equiprobable.

<!--@slideshow slide-->
## Colab demo: implementation of $n$-gram language model

In [0]:
from collections import Counter
from tqdm.notebook import tqdm

class NGramLanguageModel:
    def __init__(self, n, delta=0.001, verbose=True):
        """
        n is the parameter of the model
        Keep counters for n-grams and n-1-grams
        """
        self.n = n
        self.delta = delta
        self.n_grams_counter = Counter()  # store n-gram counts
        self.nm1_grams_counter = Counter()  # store n-1-gram counts
        self.vocab = {None}  # set of all tokens
        self.verbose = verbose  # show progressbar
    
    def fit(self, sequences):
        """
        Train the model
        """
        if self.verbose:
            sequences = tqdm(sequences, desc="fit")
        for sequence in sequences:
            # update the n-grams counter
            for n_gram in generate_n_grams(sequence, self.n):
                self.n_grams_counter[n_gram] += 1
            # update the n-1-grams counter
            for nm1_gram in generate_n_grams(sequence, self.n - 1):
                self.nm1_grams_counter[nm1_gram] += 1
            # update the vocabulary
            self.vocab |= set(sequence)
    
    def predict_token_probability(self, sequence, token):
        """
        Return P(token | sequence)
        """
        padding = [None for i in range(self.n-1)]  # add padding
        tail = (padding + sequence)[-(self.n-1):]  # get last n-1 tokens
        # estimate the conditional probability using counts with smoothing
        return (self.n_grams_counter[tuple(tail + [token])] + self.delta) / (self.nm1_grams_counter[tuple(tail)] + self.delta * len(self.vocab))

In [0]:
# read the data
import json
with open("harbour-space-text-mining-course/datasets/trump_twitter_archive/tweets.json") as f:
    tweets = json.load(f)

# prepare tokenizer
import spacy
nlp = spacy.load("en", disable=["parser", "ner", "tagger"])

# Special tokenizer for Twitter:
#  - one token for all Twitter accounts
#  - one token for all URLs
def tokenize(text):
    tokens = []
    for t in nlp(text):
        if t.text.startswith("@"):
            tokens.append("<TWITTER_ACCOUNT>")
        elif t.text.startswith("http"):
            tokens.append("<URL>")
        else:
            tokens.append(t.text)
    return tokens

In [13]:
sequences = [tokenize(tweet["text"]) for tweet in tqdm(tweets, desc="tokenize") if not tweet.get("is_retweet")]

In [14]:
lm = NGramLanguageModel(3, delta=1e-5)
lm.fit(sequences)
prefix = ["Make", "America"]
for token in ["great", "strong", "cool"]:
    print(" ".join(prefix), token, ":", lm.predict_token_probability(prefix, token))


Make America great : 0.04752055855594707
Make America strong : 0.011880184189454726
Make America cool : 5.9400623944153916e-08


<!--@slideshow slide-->
## Colab quiz 4

In [15]:
enable_mathjax_in_cell()
quiz_count_ngrams()()

<!--@slideshow slide-->
# Generate text with Language Models

The algorithm of text generation with $n$-gram language model is recursive:
1. Input: some prefix $P$ (possibly empty). Add padding to the beginning if necessary.
1. Return $P$ if the last $n-1$ symbols of $P$ are padding symbols or $P$ is too long.
1. For each token $t \in V$, compute $\Pr(t|P)$.
1. Choose $t$ at random according to probability distribution $\Pr(t|P)$.
1. Append $t$ to $P$ and repeat.

<!--@slideshow slide-->
## Colab demo: generate text with language model (implementation)

In [0]:
import numpy as np

def generate_text(language_model, prefix, seed=0, max_text_length=10):
    n = language_model.n
    generated_text = prefix[:]  # generated text starts with the given (possibly empty) prefix
    np.random.seed(seed + len(generated_text))  # new seed for each generated word

    # stopping criteria:
    # - at least one word has been generated and the generated text ends with padding
    # - or the generated text is too long
    if (len(generated_text) >= 1 and generated_text[-1] is None) or len(generated_text) > max_text_length:
        return generated_text
    # the recursive step: sample a token and add it to the prefix
    # tokens are stored in the .vocab attribute
    all_tokens = list(language_model.vocab)
    # get the probabilities for all tokens
    all_token_probabilities = np.array([
        language_model.predict_token_probability(generated_text, token)
        for token in all_tokens
    ])
    # sample
    next_token = np.random.choice(all_tokens, size=1, p=all_token_probabilities)[0]
    # generate using the updated text
    return generate_text(
        language_model,
        generated_text + [next_token],
        seed=seed,
        max_text_length=max_text_length
    )

In [17]:
for n in (2, 3, 4, 5):
    lm = NGramLanguageModel(n, delta=1e-5)
    lm.fit(sequences)
    print("n={}".format(n))
    for seed in range(3):
        generated_text = generate_text(lm, "Make America".split(), seed=seed)
        print(" ".join([token for token in generated_text if token is not None]))
    print("-" * 10)


n=2
Make America Great Again Rally <URL>
Make America !
Make America had to win in from <TWITTER_ACCOUNT> … <URL> "
----------



n=3
Make America Great Again agenda ! See you there in the
Make America Great Again !
Make America great again . ”
----------



n=4
Make America Great Again agenda ! Jobs , Jobs , Jobs
Make America Great Again !
Make America realsports Lincoln too!!"NICE 1976 circular spring foxnewsfacts notified inferiority
----------



n=5
Make America Great Again agenda ! Jobs , Jobs 225 foxnewsfacts
Make America Great realsports Lincoln too!!"NICE 1976 circular spring foxnewsfacts notified
Make America realsports Lincoln too!!"NICE 1976 circular spring foxnewsfacts notified inferiority
----------


<!--@slideshow slide-->
We can see that increasing $n$ doesn't help to generate good texts. Why?

<!--@slideshow fragment-->
**Sparsity problem**: by increasing $n$ we also increase the total number of possible $n$-grams, so counts zero out.

<!--@slideshow slide-->
## Colab demo: generate text with $n$-gram Language Model (internal details)

In [0]:
prefix = ["I", "promise"]

In [19]:
# bigram model
# the text is grammatical, but incoherent
lm = NGramLanguageModel(2, delta=1e-5)
lm.fit(sequences)
demo_generate_text_ngram(lm, prefix, seed=0)

In [20]:
# trigram model
lm = NGramLanguageModel(3, delta=1e-5)
lm.fit(sequences)
demo_generate_text_ngram(lm, prefix, seed=1)

In [21]:
# 4-gram model
# observe what happens when count() reaches 0
lm = NGramLanguageModel(4, delta=1e-5)
lm.fit(sequences)
demo_generate_text_ngram(lm, prefix, seed=0)

<!--@slideshow slide-->
# Evaluate quality of Language Model

<!--@slideshow fragment-->
**Q**: Even the simplest Language Model has 2 parameters: $n$ and $\delta$. How to choose them?


<!--@slideshow fragment-->
**A**: Choose the parameters that give the best quality!


<!--@slideshow fragment-->
**Q**: How to measure quality?

<!--@slideshow slide-->
```
Вячэрняя прахалода.
Спякотлівы дзень, бывай.
Няхай адпачне прырода,
Бадзенеўскі родны край.
```

<!--@slideshow fragment-->
If you are surprised to see it on the slide, probably your inner Language Model for Belarusian language is not so good.

<!--@slideshow slide-->

**Idea**: a language model is good for a given text if it is not "surprised" by it (it _assigns high probability_ to it).

<!--@slideshow fragment-->
Here is the formula for probability of text $t_1 t_2 \dots t_l$ (chain rule):
$$
\Pr(t_1 t_2 \dots t_l) = \Pr(t_1) \cdot \Pr(t_2 | t_1) \cdot \dots \cdot \Pr(t_l|t_1 t_2 \dots t_{l-1})
$$


<!--@slideshow fragment-->
**Problem**: longer texts have lower probability because of larger number of terms in the product.


<!--@slideshow fragment-->
**Solution**: normalize by the number of tokens.


<!--@slideshow fragment-->
**Definition**: _perplexity_ of text $t_1 t_2 \dots t_l$ is its inverse normalized probability:
$$
\textrm{Perplexity}(t_1 t_2 \dots t_l) = \dfrac{1}{\Pr(t_1 t_2 \dots t_l)^\frac{1}{l}}
$$

The lower the perplexity of the text, the better the Language Model is for the text.

In [0]:
#@slideshow slide
def perplexity(language_model, sequence):
    import numpy as np

    sum_logarithms = 0.0
    for i in range(len(sequence)):
        token = sequence[i]
        prefix = sequence[:i]
        sum_logarithms += np.log(language_model.predict_token_probability(prefix, token))
    return np.exp(-sum_logarithms / len(sequence))


<!--@slideshow slide-->
## Colab quiz 5

In [23]:
enable_mathjax_in_cell()
quiz_perplexity()()

Using perplexity, we can evaluate model's quality and choose the best model.

Perplexity is defined for a single text. To evaluate a model on a set of texts, we will average perplexity over this set.

<!--@slideshow slide-->
## Colab demo: choose the best $n$-gram language model using perplexity

In [24]:
import random
from itertools import product

# convert tweets into sequences of tokens
sequences = [tokenize(tweet["text"])
              for tweet in tqdm(tweets, desc="tokenize")
              if not tweet.get("is_retweet")
            ]
random.seed(0)
random.shuffle(sequences)
# take 30000 texts to train language models
train = sequences[:30000]
# the remaining texts are left for validation
test = sequences[30000:]
results = []

# iterate over pairs of (n, delta)
for n, delta in tqdm(list(product((1, 2, 3), (1e-1, 1e-2, 1e-3, 1e-4)))):
    # train language model
    lm = NGramLanguageModel(n, delta=delta, verbose=False)
    lm.fit(train)
    # compute average perplexity on hold-out test dataset
    avg_perplexity = sum(perplexity(lm, test_seq) for test_seq in test) / len(test)
    results.append((n, delta, avg_perplexity))

print(tabulate(results, headers=("n", "delta", "average perplexity"), floatfmt=".4f"))


  n    delta    average perplexity
---  -------  --------------------
  1   0.1000            23685.4814
  1   0.0100            23855.4823
  1   0.0010            24103.7997
  1   0.0001            24485.8272
  2   0.1000              895.3498
  2   0.0100              553.7922
  2   0.0010              583.4630
  2   0.0001              949.3447
  3   0.1000             4423.5242
  3   0.0100             2601.0637
  3   0.0010             2111.1178
  3   0.0001             2629.4928


On average, bigram models beat trigram models because trigram models suffer from sparsity problem.

However, trigram models have lower perplexity on "typical" texts:

In [25]:
typical_text = "MAKE AMERICA GREAT AGAIN!"
best_bigram_lm = NGramLanguageModel(2, delta=0.01, verbose=False)
best_bigram_lm.fit(train)
print("Best bigram model perplexity:", perplexity(best_bigram_lm, tokenize(typical_text)))
best_trigram_lm = NGramLanguageModel(3, delta=0.001, verbose=False)
best_trigram_lm.fit(train)
print("Best trigram model perplexity:", perplexity(best_trigram_lm, tokenize(typical_text)))

Best bigram model perplexity: 3.3911673707219236
Best trigram model perplexity: 1.1986171207570266


<!--@slideshow slide-->
# Text classification with language models

<!--@slideshow slide-->
**Classification problem (informally)**:
- We have a set of documents.
- Each document has a label (category).
- We need need to create an algorithm that predicts labels of **new similar** documents.

<!--@slideshow slide-->
## 20 newsgroup dataset

[This is Usenet](https://en.wikipedia.org/wiki/Usenet_newsgroup)

![Usenet](https://upload.wikimedia.org/wikipedia/commons/f/f4/Usenet_servers_and_clients.svg)

<!--@slideshow slide-->
<center>
<b>Usenet categories</b>
<table border="1">
<tbody><tr>
<td>comp.graphics<br>comp.os.ms-windows.misc<br>comp.sys.ibm.pc.hardware<br>comp.sys.mac.hardware<br>comp.windows.x</td>
<td>rec.autos<br>rec.motorcycles<br>rec.sport.baseball<br>rec.sport.hockey</td>
<td>sci.crypt<br>sci.electronics<br>sci.med<br>sci.space</td>
</tr><tr>
<td>misc.forsale</td>
<td>talk.politics.misc<br>talk.politics.guns<br>talk.politics.mideast</td>
<td>talk.religion.misc<br>alt.atheism<br>soc.religion.christian</td>
</tr>
</tbody></table>
</center>

<!--@slideshow slide-->
## Colab demo: look at the data

In [36]:
from sklearn.datasets import fetch_20newsgroups

# select a subset of categories to speed up the demonstration
categories = ("sci.space", "rec.autos", "talk.politics.misc", "comp.graphics")
# ignore metadata to avoid overfitting (for example, metadata may contain the name of target category)
remove = ('headers', 'footers', 'quotes')
dataset_train = fetch_20newsgroups(subset="train", remove=remove, categories=categories)
dataset_test = fetch_20newsgroups(subset="test", remove=remove, categories=categories)
print(type(dataset_train))
# get texts and target
texts_train = dataset_train.data
print(">>>",texts_train[:2])
y_train = dataset_train.target
print(y_train[:2])
texts_test = dataset_test.data
y_test = dataset_test.target


<class 'sklearn.utils.Bunch'>
>>> ["\n\nPlease note that there are some radiosity packages in my Resource Listing\n(under the Subject 3: FTP list)\n\nGreetings,\nNick.\n--\nNick (Nikolaos) Fotis         National Technical Univ. of Athens, Greece\nHOME: 16 Esperidon St.,       InterNet : nfotis@theseas.ntua.gr\n      Halandri, GR - 152 32   UUCP:    mcsun!ariadne!theseas!nfotis\n      Athens, GREECE          FAX: (+30 1) 77 84 578\n\nUSENET Editor of comp.graphics Resource Listing and soc.culture.greece FAQ\nNTUA/UA ACM Student Chapter Chair - we're organizing a small conference\n        in Comp. Graphics, call if you're interested to participate.", '\nI\'m sorry about your friend.  Really.  But this anecdote does nothing to\njustify the "war on drugs".  If anything, it demonstrates that the "war"\nis a miserable failure.  What it demonstrates is that people will take\ndrugs if they want to, legal or not.  Perhaps if your friend were taking\nlegal, regulated drugs under a doctors superv

In [37]:
# the easiest way to look at the data is to convert it to Pandas dataframe
import pandas as pd

# convert train
df_train = pd.DataFrame({
    "text": texts_train,
    "label": y_train,
    "category": [dataset_train.target_names[l] for l in y_train]
})

# convert test
df_test = pd.DataFrame({
    "text": texts_test,
    "label": y_test,
    "category": [dataset_test.target_names[l] for l in y_test]
})

# print the first 10 train examples
pd.set_option('display.max_colwidth', -1)  # display full texts
df_train.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,text,label,category
0,"\n\nPlease note that there are some radiosity packages in my Resource Listing\n(under the Subject 3: FTP list)\n\nGreetings,\nNick.\n--\nNick (Nikolaos) Fotis National Technical Univ. of Athens, Greece\nHOME: 16 Esperidon St., InterNet : nfotis@theseas.ntua.gr\n Halandri, GR - 152 32 UUCP: mcsun!ariadne!theseas!nfotis\n Athens, GREECE FAX: (+30 1) 77 84 578\n\nUSENET Editor of comp.graphics Resource Listing and soc.culture.greece FAQ\nNTUA/UA ACM Student Chapter Chair - we're organizing a small conference\n in Comp. Graphics, call if you're interested to participate.",0,comp.graphics
1,"\nI'm sorry about your friend. Really. But this anecdote does nothing to\njustify the ""war on drugs"". If anything, it demonstrates that the ""war""\nis a miserable failure. What it demonstrates is that people will take\ndrugs if they want to, legal or not. Perhaps if your friend were taking\nlegal, regulated drugs under a doctors supervision he might not be in the\nposition he's in now.\n\n--------------------------------------------------------------------------\n...Dale Cook ""Any town having more churches than bars has a serious\n social problem."" ---Edward Abbey\nThe opinions are mine only (i.e., they are NOT my employer's)",3,talk.politics.misc
2,"I think Mark was talking about making it available to people who didn't\nhave email in the first place.\n\nIf anybody in the Boston area wants a sci.space feed by honest-to-gosh UUCP\n(no weird offline malreaders), let me know. I'll also hand out logins to\nanyone who wants one, especially the Boston Chapter of NSS (which I keep forgetting\nto re-attend).\n\n\n",2,sci.space
3,\n,1,rec.autos
4,"THE WHITE HOUSE\n\n Office of the Press Secretary\n\n____________________________________________________________________\nFor Immediate Release April 5, 1993 \n\n REMARKS BY THE PRESIDENT\n EN ROUTE TO CAMDEN YARDS FOR ORIOLES OPENING DAY GAME\n\t \n MARC Train\n En Route to Camden Yards\n\n\n\n11:45 A.M. EDT\n\t \n\t Q\t Mr. President, what do you think of Jesse Jackson's \nprotest today?\n\t \n\t THE PRESIDENT: I think it's an informational protest. \nI think it's fine. The owners put out a statement few days ago, \nwhich they say was the first step in, you know, efforts to increase \nminority ownership and minority increases in management. I think we \nshould. I'm encouraged by Don Baylor's appointment out in Colorado. \nAnd I think it's time to make a move on that front. So, I think it's \na legitimate issue, and I think it's -- like I said, it's an \ninformational picket and not an attempt to get people not to go to \nthe game. So, I think it's good.\n\t \n\t Q\t Do you think they're moving fast enough?\n\t \n\t THE PRESIDENT: Well, I think that it was a good first \nstep. And I think you'll see some movement now. And I think it's an \nissue that deserves some attention, and they're obviously going to \ngive it some. And I think that Reverend Jackson being out there will \nhighlight the issue. So I think it's fine.\n\t \n\t Q\t Mr. President, how about the logjam in the Senate \non the economic stimulus plan? Do you think they'll be able to break \nthat and get cloture?\n\t \n\t THE PRESIDENT: I don't know, we're working at it. I \nmean, it's a classic -- there was an article in the paper today, one \nof the papers I saw, which pretty well summed it up. They said, you \nknow, this is a -- it's just a political power play. In the Senate \nthe majority does not rule. It's not like the country. It's not \nlike the -- it's not like the House. If the minority chooses, they \ncan stop majority rule. And that's what they're doing. There are a \nlot of Republican senators who have told people that they might vote \nfor the stimulus program but there's enormous partisan political \npressure not to do it. \n\t \n\t And, of course, what it means is that in this time when \nno new jobs are being created, even though there seems to be an \neconomic recovery, it means that for political p

In [38]:
print("train contains {} texts".format(df_train.shape[0]))
print("test contains {} texts".format(df_test.shape[0]))
from collections import Counter
print()
print("distribution of labels in train:")
df_train.groupby(["category"])["category"].count()

train contains 2236 texts
test contains 1489 texts

distribution of labels in train:


category
comp.graphics         584
rec.autos             594
sci.space             593
talk.politics.misc    465
Name: category, dtype: int64

<!--@slideshow slide-->
## Classification with language models

**Idea**:
- train LM for each class
- prediction:
    - for each language model compute perplexity
    - choose the model (and the corresponding class) with the lowest perplexity

<!--@slideshow fragment-->
We will evaluate the quality of classification with accuracy score:
$$
\textrm{accuracy} = \dfrac{\textrm{number of correct predictions}}{\textrm{total number of predictions}}
$$


In [0]:
from sklearn.metrics import accuracy_score

<!--@slideshow slide-->
## Colab quiz 6

In [40]:
quiz_random_benchmark()()

<!--@slideshow slide-->
## Colab demo: classification with language models

In [0]:
from collections import defaultdict
from copy import deepcopy

class LanguageModelClassifier:
    def __init__(self, n=4, delta=0.0001, pretrained_lm=None):
        self._n = n
        self._delta = delta
        # optional pre-trained language model
        self._pretrained_lm = pretrained_lm
        # store language model for each class here
        self._class2lm = dict()

    def fit(self, texts, labels):
        class2train = defaultdict(list)
        # collect separate text datasets for each class
        for text, label in zip(texts, labels):
            class2train[label].append(text)
        # fit language model for each class
        for class_, train in class2train.items():
            if self._pretrained_lm:
                # if pre-trained language model is provided, copy n-gram counts from it
                lm = deepcopy(self._pretrained_lm)
            else:
                # else train language model from scratch
                lm = NGramLanguageModel(n=self._n, delta=self._delta)
            lm.verbose = False
            # fit and save language model
            lm.fit(train)
            self._class2lm[class_] = lm
 
    def predict(self, texts):
        predictions = []
        for text in tqdm(texts, desc="predict"):
            class2perplexity = dict()
            # compute perplexity for each language model
            for class_, lm in self._class2lm.items():
                class2perplexity[class_] = perplexity(lm, text)
            # choose the class with the lowest perplexity
            class_with_lowest_perplexity = min(
                class2perplexity.items(),
                key=lambda x: x[1]
            )[0]
            predictions.append(class_with_lowest_perplexity)
        return predictions


In [0]:
# tokenize the texts
# treat any alphanumeric sequence as a token
import re
tokenized_texts_train = [re.split(r'\W+', t.lower().strip()) for t in texts_train]
tokenized_texts_test = [re.split(r'\W+', t.lower().strip()) for t in texts_test]

In [43]:
# without pretraining
clf = LanguageModelClassifier(n=2)
clf.fit(tokenized_texts_train, y_train)
print(accuracy_score(y_test, clf.predict(tokenized_texts_test)))


0.7038280725319006


<!--@slideshow slide-->
## Pre-trained Language Models
We know that language models suffer from sparsity problem. In our classification problem, there are only 2236 training examples, this may be a problem.


<!--@slideshow fragment-->
**Idea**: we can "initialize" language model with pre-training on large unlabelled dataset.

> So we don't have to train each language model for each class from scratch, we will use some initial counts from pre-trained model.

<!--@slideshow slide-->
## Colab demo: classification with pre-trained language model

In [44]:
# Let's try language model pre-trained on Reddit comments
# The code of pre-training is here: https://colab.research.google.com/drive/1DAW26wL5hxykxmLZ9W9ozk0_Otm3LpNI
import pickle
with open("harbour-space-text-mining-course/models/reddit_lm.pickle", "rb") as f:
    print("load")
    pretrained_lm = pickle.load(f)

clf2 = LanguageModelClassifier(pretrained_lm=pretrained_lm)
print("fit")
# `fit` is relatively slow because of deepcopy
clf2.fit(tokenized_texts_train, y_train)

print("predict")
print(accuracy_score(y_test, clf2.predict(tokenized_texts_test)))

load
fit
predict



0.7253190060443251


<!--@slideshow slide-->
# Summary

1. Language Model predicts
  - Probability of text
  - Probability of the next word
1. $n$-gram Language Model
  - The next word depends only on $n-1$ previous words
1. Generate text with Language Model
  - Predict probabilities for all words
  - Sample
1. Perplexity measures the quality of Language Model
  - A good model is not "surprised"
1. Text classification with Language Models:
  - Train LM for each class
  - Choose LM with smallest perplexity
1. **We can use pre-trained language models for classification.**
  - This idea is used in ULMFiT algorithm (stay tuned).

<!--@slideshow slide-->
# Recommended resources
- [📖 Language Models (nlpforhackers.io)](https://nlpforhackers.io/language-models/)
- [📖 Home page for 20 newsgroups dataset](http://qwone.com/~jason/20Newsgroups/)
- [📖 20 newsgroups dataset in sklearn](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)

<!--@slideshow slide-->
# [OPTIONAL] Perplexity as loss function
Take the logarithm of the probability of text:
$$
\log \Pr(t_1 t_2 \dots t_l) = \log \Pr(t_1) + \log \Pr(t_2 | t_1) + \dots + \log \Pr(t_l|t_1 t_2 \dots t_{l-1})
$$
Take average instead of sum:
$$
\dfrac{\log \Pr(t_1 t_2 \dots t_l)}{l} = \dfrac{\log \Pr(t_1) + \log \Pr(t_2 | t_1) + \dots + \log \Pr(t_l|t_1 t_2 \dots t_{l-1})}{l}
$$
The expression above can be interpreted as loss function (it is similar to cross entropy loss).

We are interested in minimization of the loss function, so take the previous expression with the opposite sign:
$$
\textrm{LossFunction}(t_1 t_2 \dots t_l) = -\frac{1}{l}(\log \Pr(t_1) + \log \Pr(t_2 | t_1) + \dots + \log \Pr(t_l|t_1 t_2 \dots t_{l-1}))
$$

$\exp\left(\textrm{LossFunction}(t_1 t_2 \dots t_l)\right)$ is called perplexity:
$$
\textrm{Perplexity}(t_1 t_2 \dots t_l) = \dfrac{1}{\Pr(t_1 t_2 \dots t_l)^\frac{1}{l}}
$$